In [1]:
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.internal import *
from xfem import *
from xfem.lsetcurv import *
from math import pi,e
from numpy import linspace

ngsglobals.msg_level = 2
DrawDC = MakeDiscontinuousDraw(Draw)

interactive = 1

importing ngsxfem-2.1.2504


In [2]:
# physical parameters for Biot
mu  = 1e6
lam = 1e6
# alpha = 1
alpha = 1
K = 10 # k^-1
nu = 1
# s0 = 10
s0 = 1e-2

def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(2)

# DG space order
order_eta = 2
order_u = 2
order_p = 1
# penalty parameters
beta_eta = 200
beta_u = 200
# ghost penalty parameters
gamma_s = 0.1
gamma_u = 0.1
gamma_p = 0.5
gamma_m = 0
tau_p = 0

In [3]:
eta_x = 0
eta_y = 0
u_x = IfPos(x-0.5,10*y*(y-1),-10*y*(y-1))
u_y = IfPos(y-0.5,10*x*(x-1),-10*x*(x-1))
exact_eta = CF((eta_x, eta_y))
exact_u = CF((u_x,u_y))

# 向量形式
fe = CF((0, 0))
fm = CF((0, 0))
fp = 0
etaD = exact_eta
uD = exact_u

In [4]:
# circle1 = 9/2500-(x-0.35)**2-(y-0.75)**2
circle1 = 4/625-(x-0.2)**2-(y-0.8)**2
# circle2 = 0.01-(x-0.15)**2-(y-0.45)**2
a = 0.15
b = 0.05
circle2 = 1 - (y-0.45)**2/(2*a**2) - (x-0.15)**2/(2*b**2)
a = 0.12
b = 0.06
circle3 = 1 - (x-y+0.2)**2/(2*a**2) - (x+y-1.34)**2/(2*b**2)


# 参数设置
x0, y0 = 0.7, 0.5  # 中心位置
xc = 0.1           # 焦点间距的一半，决定花生总长度
a = 0.11         # 稍微大于 xc，决定凹陷深度
theta = pi/3  # 旋转角度

# 1. 坐标变换
dx = x - x0
dy = y - y0
xr = dx * cos(theta) + dy * sin(theta)
yr = -dx * sin(theta) + dy * cos(theta)

# 2. 计算到旋转后焦点的距离平方
# 焦点在局部坐标系的 (xc, 0) 和 (-xc, 0)
d1_sq = (xr - xc)**2 + yr**2
d2_sq = (xr + xc)**2 + yr**2

# 3. 构造 Level Set (内部为正，外部为负)
# 使用 a^2 - sqrt(...) 保证量级为 L^2
circle4 = a**2 - sqrt(d1_sq * d2_sq)

circle5 = 0.01-(x-0.35)**2-(y-0.25)**2
circle6 = 4/625-(x-0.85)**2-(y-0.85)**2
a = 0.2
b = 0.05
circle7 = 1 - (x-0.7)**2/(a**2) - (y-0.15)**2/(b**2)

f1 = IfPos(circle1-circle2,circle1,circle2)
f2 = IfPos(f1-circle3,f1,circle3)
f3 = IfPos(f2-circle4, f2, circle4)
f4 = IfPos(f3-circle5, f3, circle5)
f5 = IfPos(f4-circle6, f4, circle6)
levelset = IfPos(f5-circle7, f5, circle7).Compile()

In [15]:
def solve(h0):
    square = SplineGeometry()
    square.AddRectangle((0, 0), (1, 1), bcs=('b','r','t','l'))
    ngmesh = square.GenerateMesh(maxh=h0, quad_dominated=False)
    mesh = Mesh(ngmesh)
    lsetmeshadap = LevelSetMeshAdaptation(mesh, order=1, threshold=0.1, discontinuous_qn=True)
    deformation = lsetmeshadap.CalcDeformation(levelset)
    lsetp1 = lsetmeshadap.lset_p1

    ci = CutInfo(mesh, lsetp1)
    hasneg = ci.GetElementsOfType(HASNEG)
    hasif = ci.GetElementsOfType(IF)

    # facets used for stabilization:
    ba_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasif)
    ba_surround_facets = GetElementsWithNeighborFacets(mesh,ba_facets)
    interior_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasneg)
    in_surround_facets = GetElementsWithNeighborFacets(mesh,interior_facets)

    #Unfitted DG spaces
    Ehbase = VectorL2(mesh, order=order_eta, dirichlet=[], dgjumps=True) # space for displacement
    Uhbase = VectorL2(mesh, order=order_u, dirichlet=[], dgjumps=True) # space for velocity
    Phbase = H1(mesh, order=order_p, dirichlet=[], dgjumps=True) # space for pressure 
    E = Compress(Ehbase, GetDofsOfElements(Ehbase, ci.GetElementsOfType(HASNEG)))
    U = Compress(Uhbase, GetDofsOfElements(Uhbase, ci.GetElementsOfType(HASNEG)))
    P = Compress(Phbase, GetDofsOfElements(Phbase, ci.GetElementsOfType(HASNEG)))
    fes = E*U*P
    (eta,u,p), (kxi,v,q) = fes.TnT()

    n = 1.0 / Norm(grad(lsetp1)) * grad(lsetp1)
    ne = specialcf.normal(2) # normal vectors on faces
    h = specialcf.mesh_size  
    
    strain_eta = Sym(Grad(eta))
    strain_kxi = Sym(Grad(kxi))
    mean_stress_eta = 0.5*(Stress(Sym(Grad(eta)))+Stress(Sym(Grad(eta.Other()))))*ne
    mean_stress_kxi = 0.5*(Stress(Sym(Grad(kxi)))+Stress(Sym(Grad(kxi.Other()))))*ne
    jump_eta = eta - eta.Other()
    jump_kxi = kxi - kxi.Other()
    
    
    jump_p = p - p.Other()
    jump_q = q - q.Other()
    mean_p = 0.5*(p + p.Other())
    mean_q = 0.5*(q + q.Other())
    
    
    mean_dudn = 0.5*(Grad(u)+Grad(u.Other()))*ne
    mean_dvdn = 0.5*(Grad(v)+Grad(v.Other()))*ne
    jump_u = u - u.Other()
    jump_v = v - v.Other()
    
    
    # integration domains:
    domega = dCut(lsetp1, NEG, definedonelements=hasneg, deformation=deformation)
    dk = dCut(lsetp1, NEG, skeleton=True, definedonelements=interior_facets,
              deformation=deformation)
    dsc = dCut(lsetp1, IF, definedonelements=hasif, deformation=deformation) # 在界面上的积分
    dw = dFacetPatch(definedonelements=ba_facets, deformation=deformation)
    dsbar = ds(skeleton=True,definedon=mesh.Boundaries('t|b|l|r')) # 在外边界上的积分
    dseta = ds(skeleton=True)

    Ah = BilinearForm(fes)
    # Ae
    Ah += 2*mu*InnerProduct(strain_eta,strain_kxi)*domega + lam*div(eta)*div(kxi)*domega \
            - (InnerProduct(mean_stress_eta,jump_kxi) + InnerProduct(mean_stress_kxi,jump_eta))*dk \
            - (InnerProduct(Stress(Sym(Grad(eta)))*ne,kxi) + InnerProduct(Stress(Sym(Grad(kxi)))*ne,eta))*dseta
    
    Ah +=  beta_eta/h*(2*mu*InnerProduct(jump_eta,jump_kxi)+lam*InnerProduct(jump_eta,ne)*InnerProduct(jump_kxi,ne)) * dk
    Ah += beta_eta/h*(2*mu*InnerProduct(eta,kxi) + lam*InnerProduct(eta,ne)*InnerProduct(kxi,ne))*dseta
    
    
    # order=1 i_s 
    Ah += gamma_s / (h**2) * (eta - eta.Other()) * (kxi - kxi.Other()) * dw
    
    # Be
    Ah += -alpha*(div(kxi)*p*domega - mean_p*jump_kxi*ne*dk - p*kxi*ne*dseta)
    
                  
    # Am
    Ah += nu*InnerProduct(Grad(u),Grad(v))*domega \
            - nu*(InnerProduct(mean_dudn,jump_v) + InnerProduct(mean_dvdn,jump_u) - beta_u/h*InnerProduct(jump_u,jump_v))*dk \
            - nu*(InnerProduct(Grad(u)*ne,v) + InnerProduct(Grad(v)*ne,u) - beta_u/h*InnerProduct(u,v))*dsbar\
            + K*InnerProduct(u,v)*domega
    
    
    # ghost penalty for velocity
    Ah += gamma_u / (h**2) * (u - u.Other()) * (v - v.Other()) * dw
    
    
    
     # Bm 
    Ah += -div(v)*p*domega + mean_p*jump_v*ne*dk + p*v*ne*dsbar
    
    # -Bm
    Ah += div(u)*q*domega - mean_q*jump_u*ne*dk - q*u*ne*dsbar
    
    # order=1 i_p 
    # Ah += gamma_p * (h**3) * (grad(p)*ne - grad(p.Other())*ne)*(grad(q)*ne - grad(q.Other())*ne) * dk
    Ah += gamma_p * jump_p * jump_q * dw
    
    # M
    Ah += s0*p*q*domega + gamma_m*(h**3)*(grad(p)*ne - grad(p.Other())*ne)*(grad(q)*ne - grad(q.Other())*ne) * dw
    
    # -Be
    Ah += alpha*(div(eta)*q*domega - mean_q*jump_eta*ne*dk - q*eta*ne*dseta)
    
    # stabilization
    Ah += tau_p*h*jump_p*jump_q*dk
    Ah.Assemble()
    
    
    # r.h.s
    lh = LinearForm(fes) 
    lh += fe*kxi*domega - InnerProduct(etaD,Stress(Sym(Grad(kxi)))*ne)*dseta 
    lh += beta_eta/h*(2*mu*InnerProduct(etaD,kxi) + lam*InnerProduct(etaD,ne)*InnerProduct(kxi,ne))*dseta
    lh += fm*v*domega 
    lh += - (nu*InnerProduct(uD,Grad(v)*ne) - nu*beta_u/h*uD*v)*dsbar
    lh += fp*q*domega 
    lh += - alpha*q*etaD*ne*dseta - q*uD*ne*dsbar
    lh.Assemble()

    gfu = GridFunction(fes)
    gfu.vec.data = Ah.mat.Inverse() * lh.vec
    return gfu,mesh,domega

In [16]:
gfu,mesh,domega = solve(1/100)

 Generate Mesh from spline geometry


In [12]:
mask = IfPos(levelset,0,1)
Draw(mask*gfu.components[0],mesh)


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [13]:
Draw(mask*gfu.components[1],mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [14]:
Draw(mask*gfu.components[2],mesh) 

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [ ]:
gfu2,mesh2,domega2 = solve(1/150)

 Generate Mesh from spline geometry


In [ ]:
Ehbase = VectorL2(mesh, order=order_eta, dirichlet=[], dgjumps=True) # space for displacement
Uhbase = VectorL2(mesh, order=order_u, dirichlet=[], dgjumps=True) # space for velocity
Phbase = H1(mesh, order=order_p, dirichlet=[], dgjumps=True) # space for pressure 
E = Compress(Ehbase, GetDofsOfElements(Ehbase, ci.GetElementsOfType(HASNEG)))
U = Compress(Uhbase, GetDofsOfElements(Uhbase, ci.GetElementsOfType(HASNEG)))
P = Compress(Phbase, GetDofsOfElements(Phbase, ci.GetElementsOfType(HASNEG)))
fes = E*U*P
gfu_approx = GridFunction(fes)
gfu_approx.components[0].Set(gfu2.components[0])
gfu_approx.components[1].Set(gfu2.components[1])
gfu_approx.components[2].Set(gfu2.components[2])

error_eta = sqrt(Integrate((gfu.components[0] - gfu_approx.components[0])**2* domega, mesh))
error_u = sqrt(Integrate((gfu.components[1] - gfu_approx.components[1])**2 * domega, mesh))
error_p = sqrt(Integrate((gfu.components[2] - gfu_approx.components[2])**2 * domega, mesh))